In [1]:
pip install hmmlearn

Note: you may need to restart the kernel to use updated packages.


In [81]:
import pandas as pd

pd.set_option('display.max_rows', 100)
from hmmlearn import hmm

import numpy as np
import math

In [4]:
df = pd.read_csv(r'KaggleV2-May-2016.csv')
df

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,2.572134e+12,5651768,F,2016-05-03T09:15:35Z,2016-06-07T00:00:00Z,56,MARIA ORTIZ,0,0,0,0,0,1,No
110523,3.596266e+12,5650093,F,2016-05-03T07:27:33Z,2016-06-07T00:00:00Z,51,MARIA ORTIZ,0,0,0,0,0,1,No
110524,1.557663e+13,5630692,F,2016-04-27T16:03:52Z,2016-06-07T00:00:00Z,21,MARIA ORTIZ,0,0,0,0,0,1,No
110525,9.213493e+13,5630323,F,2016-04-27T15:09:23Z,2016-06-07T00:00:00Z,38,MARIA ORTIZ,0,0,0,0,0,1,No


In [5]:
df = df.astype(dtype={"PatientId":"int64", \
                                    "ScheduledDay":"datetime64", \
                                    "AppointmentDay":"datetime64"})
df["No-show"] = df["No-show"].map({"Yes":1, "No":0}, na_action='ignore').astype("int64")
df

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,29872499824296,5642903,F,2016-04-29 18:38:08,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,0
1,558997776694438,5642503,M,2016-04-29 16:08:27,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,0
2,4262962299951,5642549,F,2016-04-29 16:19:04,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,0
3,867951213174,5642828,F,2016-04-29 17:29:31,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0
4,8841186448183,5642494,F,2016-04-29 16:07:23,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,2572134369293,5651768,F,2016-05-03 09:15:35,2016-06-07,56,MARIA ORTIZ,0,0,0,0,0,1,0
110523,3596266328735,5650093,F,2016-05-03 07:27:33,2016-06-07,51,MARIA ORTIZ,0,0,0,0,0,1,0
110524,15576631729893,5630692,F,2016-04-27 16:03:52,2016-06-07,21,MARIA ORTIZ,0,0,0,0,0,1,0
110525,92134931435557,5630323,F,2016-04-27 15:09:23,2016-06-07,38,MARIA ORTIZ,0,0,0,0,0,1,0


In [6]:
df2 = df[["PatientId","AppointmentID", "AppointmentDay", "No-show"]].sort_values(["PatientId", "AppointmentDay"])
df2.head(50)

,PatientId,AppointmentID,AppointmentDay,No-show
100517,39217,5751990,2016-06-03,0
105430,43741,5760144,2016-06-01,0
3950,93779,5712759,2016-05-18,0
73303,141724,5637648,2016-05-02,0
73228,537615,5637728,2016-05-06,0
54611,5628261,5680449,2016-05-13,1
40692,11831856,5718578,2016-05-19,0
58616,22638656,5580835,2016-05-03,0
88585,22638656,5715081,2016-06-08,0
83628,52168938,5704816,2016-05-16,0


In [97]:
df3 = df2.groupby("PatientId").apply(lambda x : x.groupby('PatientId').agg(['last']) if(x['No-show'].count()>3) else None).droplevel(level = 1).sort_index()

In [115]:
df3['AppointmentID']['last']

PatientId
251541539          5721140
422861257          5779051
575362723          5767198
724118924          5680061
934644983          5650756
                    ...   
997598781898697    5740266
998882118364464    5662038
999627467948438    5744066
999831834585814    5681860
999931985292928    5772701
Name: last, Length: 4984, dtype: int64

In [104]:
train = df2[df2['PatientId'].isin(df3.index)]
train

,PatientId,AppointmentID,AppointmentDay,No-show
47838,251541539,5618607,2016-05-13,1
16272,251541539,5692489,2016-05-19,0
47731,251541539,5737747,2016-05-25,0
108510,251541539,5721140,2016-06-01,0
1831,422861257,5638450,2016-04-29,0
...,...,...,...,...
62194,999931985292928,5616762,2016-05-04,0
32023,999931985292928,5710157,2016-05-17,0
62099,999931985292928,5700484,2016-05-17,0
32033,999931985292928,5736368,2016-05-25,0


In [116]:
train = train[~train['AppointmentID'].isin(df3['AppointmentID']['last'])]
train.head(50)

,PatientId,AppointmentID,AppointmentDay,No-show
47838,251541539,5618607,2016-05-13,1
16272,251541539,5692489,2016-05-19,0
47731,251541539,5737747,2016-05-25,0
1831,422861257,5638450,2016-04-29,0
10436,422861257,5640718,2016-05-02,0
10378,422861257,5707143,2016-05-19,0
65041,575362723,5721339,2016-05-19,0
74318,575362723,5726363,2016-05-31,0
92868,575362723,5721769,2016-06-02,1
51557,724118924,5641131,2016-05-05,0


In [117]:
temp = train.groupby("PatientId")['AppointmentID'].count()>2
unique_patient = temp[temp == True].index

In [118]:
unique_patient = train['PatientId'].unique()
unique_patient

array([      251541539,       422861257,       575362723, ...,
       999627467948438, 999831834585814, 999931985292928], dtype=int64)

In [119]:
model = hmm.GaussianHMM()

In [120]:
df3.columns

MultiIndex([( 'AppointmentID', 'last'),
            ('AppointmentDay', 'last'),
            (       'No-show', 'last')],
           )

In [ ]:
for p in unique_patient:
    print(p)
    l = np.array(train[train['PatientId']==p]['No-show']).reshape(-1, 1)
    print(l)
    model.fit(l)
    print("predict: ", model.predict(l))
    value = df3[df3.index==p]['No-show']['last']
    #print(df3[df3.index==p]['No-show']['last'])
    print("Patient: ", value, ", score: ", model.score(l))
    print("---------------------------------------------")
   

251541539
[[1]
 [0]
 [0]]


D:\Users\Magda\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'


predict:  [0 0 0]
Patient:  PatientId
251541539    0
Name: last, dtype: int64 , score:  -2.0008649355055037
---------------------------------------------
422861257
[[0]
 [0]
 [0]]


D:\Users\Magda\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Even though the 'covars_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'c'
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
D:\Users\Magda\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid i

predict:  [0 0 0]
Patient:  PatientId
422861257    0
Name: last, dtype: int64 , score:  5.798858112370283
---------------------------------------------
575362723
[[0]
 [0]
 [1]]


...

predict:  [0 0 0]
Patient:  PatientId
52499398217651    1
Name: last, dtype: int64 , score:  -2.000864935505504
---------------------------------------------
52554158156819
[[0]
 [0]
 [0]
 [0]]


Even though the 'covars_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'c'
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
D:\Users\Magda\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


predict:  [0 0 0 0]
Patient:  PatientId
52554158156819    0
Name: last, dtype: int64 , score:  8.307174961397273
---------------------------------------------
52587225747632
[[0]
 [0]
 [0]
 [1]]


Even though the 'covars_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'c'
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
D:\Users\Magda\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


predict:  [0 0 0 0]
Patient:  PatientId
52587225747632    0
Name: last, dtype: int64 , score:  -2.327975929701705
---------------------------------------------
52599251296133
[[0]
 [0]
 [0]
 [0]]


In [14]:
X1 = [[0.5], [1.0], [-1.0], [0.42], [0.24]]
X2 = [[2.4], [4.2], [0.5], [-0.24]]

In [17]:
X = np.concatenate([X1, X2])
X

array([[ 0.5 ],
       [ 1.  ],
       [-1.  ],
       [ 0.42],
       [ 0.24],
       [ 2.4 ],
       [ 4.2 ],
       [ 0.5 ],
       [-0.24]])

In [19]:
lengths = [len(X1), len(X2)]
lengths

[5, 4]

In [20]:
hmm.GaussianHMM(n_components=3).fit(X, lengths)

Fitting a model with 14 free scalar parameters with only 9 data points will result in a degenerate solution.


GaussianHMM(n_components=3)

In [ ]:
hmmm.predict(X)

https://hmmlearn.readthedocs.io/en/stable/api.html
https://medium.com/@victor.irechukwu/overview-of-the-hidden-markov-model-hmm-what-it-can-do-for-you-in-machine-learning-83b3003297b9 